In [18]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pickle
# Encode string labels to integer values
nltk.data.path.append("/path/to/nltk_data")



In [19]:
data = pd.read_csv('Student Mental Wellness Survey (Responses) - Form responses 1.csv')

data

,Timestamp,Age,Which state are you from?,Gender,Grade Level/Year in School,What are you currently studying?,How often have you been feeling sad or hopeless in the past two weeks?,"How often have you been feeling nervous, anxious, or on edge in the past two weeks?",Have you experienced changes in your appetite or weight in the past two weeks?,"Have you had trouble sleeping (either difficulty falling asleep, staying asleep, or waking up too early) in the past two weeks?",Have you been feeling unusually fatigued or lacking in energy in the past two weeks?,Have you had trouble concentrating on things such as schoolwork or other activities in the past two weeks?,Have you had recurring thoughts of self harm in the past two weeks?,"Have you experienced any physical symptoms such as racing heart, trembling, sweating, or shortness of breath without any apparent cause in the past two weeks?",Have you noticed any changes in your academic performance or attendance due to emotional distress in the past two weeks?,Is there anything else you would like to share about your emotional well-being or any concerns you have?,Do you think you have depression or anxiety?,Have you ever been diagnosed with depression or anxiety by a healthcare professional?
0,05/03/2024 22:23:50,20,Odisha,Male,Bachelor's 3rd Year,Computer Science,Everyday,Often,Weight gain,Often,Everyday,Everyday,Everyday,Rarely,No change,Nothing,Yes,No
1,05/03/2024 22:31:08,69,Liquid,Male,Bachelor's 3rd Year,Computer Science,Everyday,Everyday,No change,Everyday,Everyday,Sometimes,Everyday,Everyday,No change,nope,Yes,Yes
2,13/03/2024 12:53:08,20,Hariyana,Male,Bachelor's 3rd Year,BBA,Sometimes,Rarely,No change,No,No,No,No,No,No change,No,No,Yes
3,13/03/2024 12:54:03,21,UP,Male,Bachelor's 4th Year,CSE,No,No,No change,No,No,No,No,No,No change,NaN,Yes,Yes
4,13/03/2024 12:54:48,56,Delhi,Female,Bachelor's 2nd Year,BA,Rarely,Rarely,Increased appetite,Rarely,Rarely,Rarely,Rarely,Rarely,Slight decline,NaN,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,28/03/2024 11:39:31,20,Punjab,Female,Bachelor's 3rd Year,Bsc IT,Sometimes,Sometimes,No change,Sometimes,Rarely,Sometimes,No,Sometimes,No change,No,I am not sure.,No
178,28/03/2024 11:42:48,20,Maharashtra,Female,Bachelor's 3rd Year,Bcom accounting and finance,Often,Sometimes,No change,Rarely,No,Rarely,No,No,Significant decline,NaN,I am not sure.,No
179,28/03/2024 16:24:49,19,Maharashtra,Female,Bachelor's 2nd Year,Bsc nursing,No,No,Increased appetite,No,Sometimes,Rarely,No,No,No change,None i am good,No,No
180,28/03/2024 18:26:02,18,maharashtra,Female,Bachelor's 1st Year,fashion,Sometimes,Often,Decreased appetite,Often,Rarely,Sometimes,No,Sometimes,No change,NaN,Yes,No


In [20]:
data.columns = [f'X{i}' for i in range(1, len(data.columns) + 1)]
data = data.iloc[:, 1:]
data

,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18
0,20,Odisha,Male,Bachelor's 3rd Year,Computer Science,Everyday,Often,Weight gain,Often,Everyday,Everyday,Everyday,Rarely,No change,Nothing,Yes,No
1,69,Liquid,Male,Bachelor's 3rd Year,Computer Science,Everyday,Everyday,No change,Everyday,Everyday,Sometimes,Everyday,Everyday,No change,nope,Yes,Yes
2,20,Hariyana,Male,Bachelor's 3rd Year,BBA,Sometimes,Rarely,No change,No,No,No,No,No,No change,No,No,Yes
3,21,UP,Male,Bachelor's 4th Year,CSE,No,No,No change,No,No,No,No,No,No change,NaN,Yes,Yes
4,56,Delhi,Female,Bachelor's 2nd Year,BA,Rarely,Rarely,Increased appetite,Rarely,Rarely,Rarely,Rarely,Rarely,Slight decline,NaN,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,20,Punjab,Female,Bachelor's 3rd Year,Bsc IT,Sometimes,Sometimes,No change,Sometimes,Rarely,Sometimes,No,Sometimes,No change,No,I am not sure.,No
178,20,Maharashtra,Female,Bachelor's 3rd Year,Bcom accounting and finance,Often,Sometimes,No change,Rarely,No,Rarely,No,No,Significant decline,NaN,I am not sure.,No
179,19,Maharashtra,Female,Bachelor's 2nd Year,Bsc nursing,No,No,Increased appetite,No,Sometimes,Rarely,No,No,No change,None i am good,No,No
180,18,maharashtra,Female,Bachelor's 1st Year,fashion,Sometimes,Often,Decreased appetite,Often,Rarely,Sometimes,No,Sometimes,No change,NaN,Yes,No


In [21]:
data['X3'].unique()

array(['Odisha', 'Liquid', 'Hariyana ', 'UP ', 'Delhi', 'Rajisthan',
       'haryana', 'Bihar', 'Up', 'Uttar pradesh ', 'Uttar Pradesh ',
       'Maharashtra ', 'Bihar ', 'Uttar pradesh', 'West Bengal ',
       'Maharastra ', '-', 'Madhya Pradesh ', 'Jharkhand ', 'Tamil Nadu ',
       'UP', 'Madhya pradesh ', 'Uttar Pradesh', 'Haryana', 'Odisha ',
       'Jharkhand', 'Rajasthan', 'Chhattisgarh', 'HARYANA ',
       'Chhattisgarh ', 'MP', 'Karnataka ', 'Maharashtra',
       'Not from India', 'West Bengal', 'Gujrat ', 'Delhi NCR',
       'Madhya Pradesh', 'UttarPradesh', 'uttar Pradesh', 'Assam',
       'maharashtra ', 'Karnataka', 'Goa', 'delhi ', 'mp', 'Mahrashtra',
       'Punjab'], dtype=object)

In [22]:
data.loc[:, 'X3'] = data['X3'].str.strip()
data['X3'].unique()

array(['Odisha', 'Liquid', 'Hariyana', 'UP', 'Delhi', 'Rajisthan',
       'haryana', 'Bihar', 'Up', 'Uttar pradesh', 'Uttar Pradesh',
       'Maharashtra', 'West Bengal', 'Maharastra', '-', 'Madhya Pradesh',
       'Jharkhand', 'Tamil Nadu', 'Madhya pradesh', 'Haryana',
       'Rajasthan', 'Chhattisgarh', 'HARYANA', 'MP', 'Karnataka',
       'Not from India', 'Gujrat', 'Delhi NCR', 'UttarPradesh',
       'uttar Pradesh', 'Assam', 'maharashtra', 'Goa', 'delhi', 'mp',
       'Mahrashtra', 'Punjab'], dtype=object)

In [23]:
data.loc[:, 'X3'] = data['X3'].str.strip().str.lower()
replacements = {
    'uttar pradesh': 'Uttar Pradesh',
    'up': 'Uttar Pradesh',
    'uttarpradesh': 'Uttar Pradesh',
    'madhya pradesh': 'Madhya Pradesh',
    'mp': 'Madhya Pradesh',
    'maharashtra': 'Maharashtra',
    'mahrashtra': 'Maharashtra',
    'haryana': 'Haryana',
    'hariyana': 'Haryana',
    'odisha': 'Odisha',
    'west bengal': 'West Bengal',
    'chhattisgarh': 'Chhattisgarh',
    'karnataka': 'Karnataka',
    'delhi': 'Delhi',
    'delhi ncr': 'Delhi',
    'rajasthan': 'Rajasthan',
    'rajisthan': 'Rajasthan',
    'tamil nadu': 'Tamil Nadu', 
    'gujrat': 'Gujarat',
    'maharastra': 'Maharashtra'
}

data.loc[:, 'X3'] = data['X3'].replace(replacements)

data.loc[:, 'X3'] = data['X3'].apply(lambda x: x.title())

indian_states_ut = [
    'Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chhattisgarh', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jharkhand', 'Karnataka', 'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal', 'Andaman and Nicobar Islands', 'Chandigarh', 'Dadra and Nagar Haveli and Daman and Diu', 'Delhi', 'Lakshadweep', 'Puducherry'
]


data['X3'] = data['X3'].apply(lambda x: x if x in indian_states_ut else 'Others')

# Print value counts
data['X3'].value_counts()

X3
Odisha            36
Maharashtra       35
Madhya Pradesh    19
Bihar             18
Jharkhand         18
Uttar Pradesh     14
Haryana            7
Assam              7
Delhi              6
West Bengal        6
Chhattisgarh       4
Others             3
Karnataka          3
Rajasthan          2
Tamil Nadu         1
Gujarat            1
Goa                1
Punjab             1
Name: count, dtype: int64

In [24]:
data['X6'] = data['X6'].str.strip()

# Checking unique values after removing extra spaces
unique_values = data['X6'].unique()
print(unique_values)


['Computer Science' 'BBA' 'CSE' 'BA' 'Senior High School' 'B.tech' 'BTech'
 'Engeneering' 'BTech-CSE' 'B.Tech.' 'B.Tech' 'Teacher' 'Computer science'
 'B.Tech computer science' 'Btech' 'B.tech,CSE' 'B tech'
 'KIIT BHUBANESWAR' 'BTECH - CSE' 'React native, Machine learning'
 'Computer Science Engineering' 'KIIT' 'BTECH IN CSE' 'B. Tech.'
 "Bachelor's in design" 'School student' 'BFA' 'Political Science Honours'
 'Computer science engineering' 'BTech in CSE' 'B Tech CSE' 'B-tech'
 'B. Tech'
 'International relations with defence and strategic Studies Research'
 'B. Tech CSE' 'BTech in cse' 'Biotechnology'
 'Computer Science and Engineering' 'BA LLB'
 'Animation and interior design' 'B tech, electrical engineering' 'Bcom'
 'Chartered Accountancy' 'Mechanical Engineering'
 'B.Tech. in Computer Science and Engineering' 'Information Technology'
 'Engineering' 'B.Com hons (marketing) with Liberal Arts' 'BAMS'
 'B.Tech. in Artificial Intelligence and Data Science'
 'BA Hons. - Psychology' 'BCo

In [25]:
category_mapping = {
    'Computer Science': 'Computer Science',
    'CSE': 'Computer Science',
    'B.tech': 'Engineering',
    'BTech': 'Engineering',
    'B.Tech.': 'Engineering',
    'B.Tech': 'Engineering',
    'BTech-CSE': 'Computer Science',
    'Engeneering': 'Engineering',
    'Teacher': 'Others',
    'KIIT BHUBANESWAR': 'Computer Science',
    'BTECH - CSE': 'Computer Science',
    'React native, Machine learning': 'Computer Science',
    'Computer Science Engineering': 'Computer Science',
    'KIIT': 'Others',
    'BTECH IN CSE': 'Computer Science',
    'B. Tech.': 'Engineering',
    "Bachelor's in design": 'Design',
    'School student': 'Others',
    'BFA': 'Design',
    'Political Science Honours': 'Others',
    'Biotechnology': 'Others',
    'CSE': 'Computer Science',
    'BA LLB': 'Others',
    'Animation and interior design': 'Design',
    'B tech, electrical engineering': 'Engineering',
    'Bcom': 'Commerce',
    'Chartered Accountancy': 'Others',
    'Mechanical Engineering': 'Engineering',
    'Information Technology': 'Engineering',
    'Engineering': 'Engineering',
    'B.Com hons (marketing) with Liberal Arts': 'Commerce',
    'BAMS': 'Medical',
    'B.Tech. in Artificial Intelligence and Data Science': 'Engineering',
    'BA Hons. - Psychology': 'Psychology',
    'BCom Honors': 'Commerce',
    'BA Psychology Honors': 'Psychology',
    'B.tech CS (IOT - CYBERSECURITY)': 'Computer Science',
    'BCA': 'Others',
    'Btech in Computer Science Engineering': 'Computer Science',
    'Graphic Design': 'Design',
    'Psychology': 'Psychology',
    'Architecture': 'Design',
    'Studied architecture': 'Design',
    'BTech CSE': 'Computer Science',
    'Preparing For Civil Services !!': 'Others',
    'Masters in economics': 'Others',
    'B.TECH': 'Engineering',
    'btech cse': 'Computer Science',
    'Btech(CSE)': 'Computer Science',
    'Finance in Mba': 'MBA',
    'B.com hons.': 'Commerce',
    'B com': 'Commerce',
    'BTECH': 'Engineering',
    'No': 'Others',
    'Commerce': 'Commerce',
    'Bsc Computer Science': 'Computer Science',
    'BSc. Applied Psychology': 'Psychology',
    'Bdes': 'Design',
    'Bachelor’s of Applied IT': 'Others',
    'Books': 'Others',
    'Design': 'Design',
    '-': 'Others',
    'Neet': 'Medical',  
    'Fashion design': 'Fashion',
    'B.Des': 'Design',
    'Bachelors in Dental Surgery': 'Medical',  
    'Math': 'Others',
    'B. Tech': 'Engineering',
    'Bachelors of Technology in Computer Science': 'Computer Science',
    'IITBOMBAY': 'Engineering',  
    'Bsc': 'BSc',  
    'BTech Aerospace Engineering': 'Engineering',
    'btech aerospace': 'Engineering',
    'MBA': 'MBA',
    'Film studies': 'Others',
    'B.tech aerospace engineering': 'Engineering',
    'BTech Aerospace': 'Engineering',
    'Aerospace Engineering': 'Engineering',
    'MBA - Operations and SCM': 'MBA',
    'B-tech': 'Engineering',
    'Mathematics and computing': 'Others',
    'In job': 'Others',
    'B.Tech CSE': 'Computer Science',
    'WINDCHILL+': 'Others',
    'computer': 'Others',
    'Bsc IT': 'BSc',
    'Bcom accounting and finance': 'Commerce',
    'Bsc nursing': 'Medical',
    'fashion': 'Fashion',
    'Computer science': 'Computer Science',
    'Computer Science and Engineering': 'Computer Science',
    'Btech': 'Engineering',
    'B tech': 'Engineering',
    'B. Tech': 'Engineering',
    'Btech cse': 'Computer Science',
    'B.Tech CSE': 'Computer Science',
    'Computer Science and Communication Engineering': 'Computer Science',
    'BSc Computer Science': 'Computer Science',
    'Btech CSE': 'Computer Science',
    'Btech in cse': 'Computer Science',
    'B tech aerospace': 'Engineering',
    'B.tech aerospace engineering': 'Engineering',
    'BTech Aerospace': 'Engineering',
    'Aerospace Engineering': 'Engineering',
    'Btech Aerospace': 'Engineering',
    'B. Tech in Computer Science and Engineering': 'Computer Science',
    'B tech, electrical engineering': 'Engineering',
    'Bcom accounting and finance': 'Commerce',
    'Bcom hons.': 'Commerce',
    'B com': 'Commerce',
    'B.Com hons (marketing) with Liberal Arts': 'Commerce',
    'MBA - Operations and SCM': 'MBA',
    'Masters in economics': 'Others',
    'Fashion design': 'Fashion',
    'Studied architecture': 'Design',
    'Architecture': 'Design',
    'Graphic Design': 'Design',
    'Animation and interior design': 'Design',
    'Bachelor’s of Applied IT': 'Others',
    'Biotechnology': 'Others',
    'BAMS': 'Medical',
    'Bachelors in Dental Surgery': 'Medical',
    'BSc. Applied Psychology': 'Psychology',
    'BA Hons. - Psychology': 'Psychology',
    'Computer': 'Others',
    'Math': 'Others',
    'No': 'Others',
    'B. Tech CSE': 'Computer Science',
    'Computer science engineering': 'Computer Science',
    'BTech in CSE': 'Computer Science',
    "Bachelor's of design": 'Design',
    'BTech(CSE)': 'Computer Science',
    'International relations with defence and strategic Studies Research': 'Others',
    'B Tech CSE': 'Computer Science',
    'BA': 'Others',
    'B.Tech. in Computer Science and Engineering': 'Computer Science',
    'B.Tech computer science': 'Computer Science',
    'Senior High School': 'Others',
    'BBA': 'Others',
    'B.tech,CSE': 'Computer Science',
    'B.tech Aerospace Engineering': 'Engineering',
    'Bachelors in Design ( Animation, vfx and Illustration)': 'Design',
    'Computer science and engineering' : 'Computer Science', 'BTech in cse':'Computer Science'
}


In [26]:

missing_mappings = set(unique_values) - set(category_mapping.keys())
print("Missing mappings:")
print(missing_mappings)


Missing mappings:
set()


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   X2      182 non-null    int64 
 1   X3      182 non-null    object
 2   X4      182 non-null    object
 3   X5      182 non-null    object
 4   X6      182 non-null    object
 5   X7      182 non-null    object
 6   X8      182 non-null    object
 7   X9      182 non-null    object
 8   X10     182 non-null    object
 9   X11     182 non-null    object
 10  X12     182 non-null    object
 11  X13     182 non-null    object
 12  X14     182 non-null    object
 13  X15     182 non-null    object
 14  X16     89 non-null     object
 15  X17     182 non-null    object
 16  X18     182 non-null    object
dtypes: int64(1), object(16)
memory usage: 24.3+ KB


In [28]:
# Assuming 'data' is your DataFrame containing the 'X6' column
data['X6'] = data['X6'].map(category_mapping)


data['X6'].value_counts()


X6
Computer Science    60
Engineering         59
Others              24
Design              12
Commerce            11
Medical              4
Psychology           4
MBA                  3
BSc                  3
Fashion              2
Name: count, dtype: int64

In [29]:
data

,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18
0,20,Odisha,Male,Bachelor's 3rd Year,Computer Science,Everyday,Often,Weight gain,Often,Everyday,Everyday,Everyday,Rarely,No change,Nothing,Yes,No
1,69,Others,Male,Bachelor's 3rd Year,Computer Science,Everyday,Everyday,No change,Everyday,Everyday,Sometimes,Everyday,Everyday,No change,nope,Yes,Yes
2,20,Haryana,Male,Bachelor's 3rd Year,Others,Sometimes,Rarely,No change,No,No,No,No,No,No change,No,No,Yes
3,21,Uttar Pradesh,Male,Bachelor's 4th Year,Computer Science,No,No,No change,No,No,No,No,No,No change,NaN,Yes,Yes
4,56,Delhi,Female,Bachelor's 2nd Year,Others,Rarely,Rarely,Increased appetite,Rarely,Rarely,Rarely,Rarely,Rarely,Slight decline,NaN,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,20,Punjab,Female,Bachelor's 3rd Year,BSc,Sometimes,Sometimes,No change,Sometimes,Rarely,Sometimes,No,Sometimes,No change,No,I am not sure.,No
178,20,Maharashtra,Female,Bachelor's 3rd Year,Commerce,Often,Sometimes,No change,Rarely,No,Rarely,No,No,Significant decline,NaN,I am not sure.,No
179,19,Maharashtra,Female,Bachelor's 2nd Year,Medical,No,No,Increased appetite,No,Sometimes,Rarely,No,No,No change,None i am good,No,No
180,18,Maharashtra,Female,Bachelor's 1st Year,Fashion,Sometimes,Often,Decreased appetite,Often,Rarely,Sometimes,No,Sometimes,No change,NaN,Yes,No


In [30]:
data['X16'] = data['X16'].astype(str)

sia = SentimentIntensityAnalyzer()
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()


negative_responses = ['no', 'nope', 'nothing', 'na','nope.','nah','not at all','no.', ]


data['X16_sentiment'] = data['X16'].apply(lambda x: 0.0 if any(token in [stemmer.stem(lemmatizer.lemmatize(word)) for word in nltk.word_tokenize(x.lower())] for token in negative_responses) else sia.polarity_scores(x)['compound'] if isinstance(x, str) else 0.0)

columns_to_encode = [col for col in data.columns if col not in ['X2','X17', 'X16', 'X16_sentiment','X18']]
data_encoded = pd.get_dummies(data, columns=columns_to_encode, drop_first=False)

data = data_encoded.drop(columns=['X16'])
data


,X2,X17,X18,X16_sentiment,X3_Assam,X3_Bihar,X3_Chhattisgarh,X3_Delhi,X3_Goa,X3_Gujarat,...,X13_Rarely,X13_Sometimes,X14_Everyday,X14_No,X14_Often,X14_Rarely,X14_Sometimes,X15_No change,X15_Significant decline,X15_Slight decline
0,20,Yes,No,0.0000,False,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,False
1,69,Yes,Yes,0.0000,False,False,False,False,False,False,...,False,False,True,False,False,False,False,True,False,False
2,20,No,Yes,0.0000,False,False,False,False,False,False,...,False,False,False,True,False,False,False,True,False,False
3,21,Yes,Yes,0.0000,False,False,False,False,False,False,...,False,False,False,True,False,False,False,True,False,False
4,56,No,No,0.0000,False,False,False,True,False,False,...,True,False,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,20,I am not sure.,No,0.0000,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False
178,20,I am not sure.,No,0.0000,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False
179,19,No,No,-0.3412,False,False,False,False,False,False,...,False,False,False,True,False,False,False,True,False,False
180,18,Yes,No,0.0000,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False


In [31]:
def merge_columns(row):
    if row['X18'] == 'Yes':
        return True
    elif row['X17'] == 'Yes':
        return True
    else: 
        return False


data['Output'] = data_encoded.apply(merge_columns, axis=1)
data.drop(['X18', 'X17'], axis=1, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 96 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   X2                         182 non-null    int64  
 1   X16_sentiment              182 non-null    float64
 2   X3_Assam                   182 non-null    bool   
 3   X3_Bihar                   182 non-null    bool   
 4   X3_Chhattisgarh            182 non-null    bool   
 5   X3_Delhi                   182 non-null    bool   
 6   X3_Goa                     182 non-null    bool   
 7   X3_Gujarat                 182 non-null    bool   
 8   X3_Haryana                 182 non-null    bool   
 9   X3_Jharkhand               182 non-null    bool   
 10  X3_Karnataka               182 non-null    bool   
 11  X3_Madhya Pradesh          182 non-null    bool   
 12  X3_Maharashtra             182 non-null    bool   
 13  X3_Odisha                  182 non-null    bool   

In [32]:
data.to_csv('encoded_data.csv', index=False)

In [33]:
X = data.drop(columns=['Output'])  # Features
y = data['Output']  
# Assuming X and y are your feature matrix and target vector respectively
# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Encode the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Standardize your features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define your neural network architecture
nn_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')  # Output layer with softmax activation for multiclass classification
])

# Compile the model
nn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the neural network
nn_model.fit(X_train_scaled, y_train_encoded, epochs=300, batch_size=50, validation_split=0.2)  # Adjust epochs, batch_size, and validation_split as needed

# Evaluate the model
test_loss, test_accuracy = nn_model.evaluate(X_test_scaled, y_test_encoded)
print("Test Accuracy:", test_accuracy)



c:\Ananconda\envs\SELF\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 292ms/step - accuracy: 0.4123 - loss: 1.1055 - val_accuracy: 0.4483 - val_loss: 1.3458
Epoch 2/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.6251 - loss: 0.8940 - val_accuracy: 0.4828 - val_loss: 1.1786
Epoch 3/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.6784 - loss: 0.7440 - val_accuracy: 0.5172 - val_loss: 1.0600
Epoch 4/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 123ms/step - accuracy: 0.7829 - loss: 0.6389 - val_accuracy: 0.5517 - val_loss: 0.9805
Epoch 5/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.7897 - loss: 0.5674 - val_accuracy: 0.5517 - val_loss: 0.9383
Epoch 6/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.8195 - loss: 0.4980 - val_accuracy: 0.5517 - val_loss: 0.9186
Epoch 7/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.8760 - loss: 0.4331 - val_accuracy: 0.5862 - val_loss: 0.9152
Epoch 8/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 160ms/step - accuracy: 0.8803 - loss: 0.3897 - val_accuracy: 0.5517 - va